# High-level SQL API

In [ ]:
!lndb init --storage sqlapi

In [ ]:
import sqlmodel as sqm
import pandas as pd
from datetime import datetime
from typing import Union, Tuple, Optional

In [ ]:
import lamindb as ln
from lamindb.dev.db._select import select

In [ ]:
class tester(sqm.SQLModel, table=True):
    id: int = sqm.Field(primary_key=True)
    name: str
    team_id: Optional[int] = sqm.Field(foreign_key="team.id")
    created_at: datetime = sqm.Field(default_factory=datetime.utcnow)

In [ ]:
class team(sqm.SQLModel, table=True):
    id: int = sqm.Field(primary_key=True)
    name: str

Trick lamindb setup.

In [ ]:
!rm sqlapi/sqlapi.lndb

In [ ]:
engine = sqm.create_engine("sqlite:///sqlapi/sqlapi.lndb")

In [ ]:
sqm.SQLModel.metadata.create_all(engine)

In [ ]:
with sqm.Session(engine) as session:
    team_1 = team(id=0, name="stars")
    session.add(team_1)
    team_2 = team(id=1, name="moons")
    session.add(team_2)
    session.add(tester(id=0, name="test1", team_id=team_1.id))
    session.add(tester(id=1, name="test2", team_id=team_1.id))
    session.add(tester(id=2, name="test3"))
    session.commit()

In [ ]:
select(tester, id=0).one()

In [ ]:
select(tester).df()

In [ ]:
select(tester, team_id=0).all()

In [ ]:
select(tester).where(tester.name == "test1").df()

In [ ]:
select(tester).where(tester.name == "test1").df()

In [ ]:
select(tester).where(tester.id == 0, tester.name == "test1").df()

In [ ]:
select(tester).where(tester.id == 0, tester.created_at <= datetime.utcnow()).df()

In [ ]:
select(tester).where(sqm.or_(tester.name == "test1", tester.name == "test2")).df()

In [ ]:
select(tester).where(sqm.or_(tester.name == "test1", tester.name == "test2")).offset(
    1
).df()

In [ ]:
select(tester).where(sqm.or_(tester.name == "test1", tester.name == "test2")).offset(
    0
).limit(1).df()

In [ ]:
select(tester).where(sqm.or_(tester.name == "test1", tester.name == "test2")).order_by(
    sqm.desc(tester.created_at)
).df()

In [ ]:
select(tester).join(team).df()

In [ ]:
select(tester, team).join(team).all()

In [ ]:
dfs = select(tester, team).join(team).df()

In [ ]:
for df in dfs:
    display(df)

In [ ]:
select(tester, team).where(tester.team_id == team.id).all()

Clean up.

In [ ]:
ln.settings.instance._sqlite_file.unlink()